# Topic 2: Recommendation System
### Demo: Underthesea, Gemsim, Cosin Similarity

In [2]:
!pip install gensim
# !pip install underthesea

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
# from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
from gensim import corpora, models, similarities
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [4]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

%cd '/content/gdrive/MyDrive/LDS0/Khoa_304/Demo_project3'

Mounted at /content/gdrive
/content/gdrive/MyDrive/LDS0/Khoa_304/Demo_project3


In [5]:
# Dữ liệu thử nghiệm
df= pd.read_csv('samples_17062025.csv')

In [6]:
df.head()

,id,Company Name,Company Type,Company industry,Company size,Country,Working days,Overtime Policy,Company overview,Our key skills,Why you'll love working here,Location,Href
0,68,Cloud Technology,IT Outsourcing,IT Services and IT Consulting,1-50 employees,Vietnam,Monday - Friday,No OT,Cloud Technology VN\nWe’re a group of entrepre...,Skills We Use frequently\nJava\nNodeJS\nReactJ...,Friendly-Relaxed environment\nBe a part of fas...,"{'Ho Chi Minh': ['341-343, Dien Bien Phu - Dmc...",https://itviec.com/companies/cloud-technology\n
1,12,Alpaca Vietnam,IT Product,Software Products and Web Services,51-150 employees,Vietnam,Monday - Friday,No OT,IT Experts in Insurance Domain\nAlpaca Vietnam...,Our main tech stack\nReactJS\nJava\n.NET\nTest...,Attractive salary package\nSecond home with cl...,"{'Ha Noi': ['23/28 Nguyen Hong, Lang Ha, Dong ...",https://itviec.com/companies/alpaca-vietnam\n
2,325,OMNIA SYSTEMS COMPANY LIMITED,IT Product,IT Services and IT Consulting,1-50 employees,Sweden,Monday - Friday,No OT,"Omnia Systems Vietnam, the top-ranked digital ...",.NET\nJavaScript\nASP.NET\nReact Native\nC#\nT...,Attractive salary and benefits\nFlexible Worki...,"{'Ho Chi Minh': ['5th floor, 186 Dien Bien Phu...",https://itviec.com/companies/omnia-systems-com...
3,230,Innotech Vietnam Corporation,IT Outsourcing,Software Development Outsourcing,51-150 employees,United States,Monday - Friday,No OT,The top Fintech software outsourcing company f...,"Java, Mobile\nSoftware Architect\nQA QC\nJava\...",Highly competitive Salary & Benefits\nBest per...,{'Others': ['2042 Nichole Woods Drive Houston ...,https://itviec.com/companies/innotech-vietnam-...
4,113,Công ty TNHH Icetea Software,IT Outsourcing,Software Development Outsourcing,51-150 employees,Vietnam,Monday - Friday,No OT,Icetea Software\nIcetea Software is an innovat...,NaN,NaN,"{'Ha Noi': ['37 Lê Văn Lương, tòa HUD, Thanh X...",https://itviec.com/companies/cong-ty-tnhh-icet...


In [7]:
# Hàm loại bỏ stopwords dùng thư viện cho tiếng Anh
def remove_stopwords_sklearn(text):
    if pd.isna(text):
        return ""
    words = re.findall(r'\b\w+\b', text.lower())
    tokens = [word for word in words if word not in ENGLISH_STOP_WORDS]
    return ' '.join(tokens)

# Tạo cột mới
df['Company_overview'] = df['Company overview'].apply(remove_stopwords_sklearn)

In [8]:
# Tạo nội dung tổng hợp
# Lựa chọn các thông tin cần thiết để đưa vào phần content (giúp đề xuất nội dung)
# Có thể 1 cột hoặc nhiều cột thông tin
# ví dụ ở đây là chọn 1 cột, và chỉ lấy 300 từ đầu tiên trong phần mô tả (dễ quan sát khi làm demo)
df['Content'] = df['Company_overview'].apply(lambda x: ' '.join(x.split()[:300]))

In [9]:
df.head()

,id,Company Name,Company Type,Company industry,Company size,Country,Working days,Overtime Policy,Company overview,Our key skills,Why you'll love working here,Location,Href,Company_overview,Content
0,68,Cloud Technology,IT Outsourcing,IT Services and IT Consulting,1-50 employees,Vietnam,Monday - Friday,No OT,Cloud Technology VN\nWe’re a group of entrepre...,Skills We Use frequently\nJava\nNodeJS\nReactJ...,Friendly-Relaxed environment\nBe a part of fas...,"{'Ho Chi Minh': ['341-343, Dien Bien Phu - Dmc...",https://itviec.com/companies/cloud-technology\n,cloud technology vn group entrepreneurs usa fr...,cloud technology vn group entrepreneurs usa fr...
1,12,Alpaca Vietnam,IT Product,Software Products and Web Services,51-150 employees,Vietnam,Monday - Friday,No OT,IT Experts in Insurance Domain\nAlpaca Vietnam...,Our main tech stack\nReactJS\nJava\n.NET\nTest...,Attractive salary package\nSecond home with cl...,"{'Ha Noi': ['23/28 Nguyen Hong, Lang Ha, Dong ...",https://itviec.com/companies/alpaca-vietnam\n,experts insurance domain alpaca vietnam establ...,experts insurance domain alpaca vietnam establ...
2,325,OMNIA SYSTEMS COMPANY LIMITED,IT Product,IT Services and IT Consulting,1-50 employees,Sweden,Monday - Friday,No OT,"Omnia Systems Vietnam, the top-ranked digital ...",.NET\nJavaScript\nASP.NET\nReact Native\nC#\nT...,Attractive salary and benefits\nFlexible Worki...,"{'Ho Chi Minh': ['5th floor, 186 Dien Bien Phu...",https://itviec.com/companies/omnia-systems-com...,omnia systems vietnam ranked digital workplace...,omnia systems vietnam ranked digital workplace...
3,230,Innotech Vietnam Corporation,IT Outsourcing,Software Development Outsourcing,51-150 employees,United States,Monday - Friday,No OT,The top Fintech software outsourcing company f...,"Java, Mobile\nSoftware Architect\nQA QC\nJava\...",Highly competitive Salary & Benefits\nBest per...,{'Others': ['2042 Nichole Woods Drive Houston ...,https://itviec.com/companies/innotech-vietnam-...,fintech software outsourcing company focusing ...,fintech software outsourcing company focusing ...
4,113,Công ty TNHH Icetea Software,IT Outsourcing,Software Development Outsourcing,51-150 employees,Vietnam,Monday - Friday,No OT,Icetea Software\nIcetea Software is an innovat...,NaN,NaN,"{'Ha Noi': ['37 Lê Văn Lương, tòa HUD, Thanh X...",https://itviec.com/companies/cong-ty-tnhh-icet...,icetea software icetea software innovative out...,icetea software icetea software innovative out...


In [10]:
df[["Content"]].head(2)

,Content
0,cloud technology vn group entrepreneurs usa fr...
1,experts insurance domain alpaca vietnam establ...


In [11]:
import re

In [12]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Vector hóa nội dung
vectorizer = TfidfVectorizer(analyzer='word')
tfidf_matrix = vectorizer.fit_transform(df['Content'])

# Tính toán độ tương đồng
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [13]:
df_show = pd.DataFrame(cosine_sim)
df_show

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.000000,0.039289,0.080596,0.058722,0.041211,0.119373,0.046882,0.063833,0.079022,0.000000,0.024797,0.073282,0.052967,0.088745,0.059259,0.058445,0.014351,0.019723,0.0,0.035561
1,0.039289,1.000000,0.084235,0.074692,0.079207,0.044908,0.078965,0.032191,0.091088,0.013390,0.035718,0.052346,0.070191,0.123778,0.000000,0.053426,0.062373,0.030013,0.0,0.036063
2,0.080596,0.084235,1.000000,0.039529,0.028909,0.059343,0.011291,0.021812,0.017989,0.027351,0.048392,0.064699,0.035547,0.062916,0.040080,0.083374,0.024935,0.033281,0.0,0.076581
3,0.058722,0.074692,0.039529,1.000000,0.190307,0.106266,0.083680,0.205096,0.159155,0.159606,0.067118,0.085705,0.199344,0.121137,0.040167,0.066412,0.050115,0.018209,0.0,0.021622
4,0.041211,0.079207,0.028909,0.190307,1.000000,0.095176,0.040645,0.016452,0.136409,0.081145,0.036228,0.045185,0.119769,0.074857,0.015446,0.034377,0.010888,0.045303,0.0,0.041102
5,0.119373,0.044908,0.059343,0.106266,0.095176,1.000000,0.028653,0.094633,0.062169,0.055146,0.025246,0.062207,0.071579,0.111257,0.061644,0.082017,0.010822,0.030424,0.0,0.049666
6,0.046882,0.078965,0.011291,0.083680,0.040645,0.028653,1.000000,0.103937,0.078656,0.067860,0.037683,0.033877,0.038583,0.087698,0.034142,0.025737,0.048809,0.011331,0.0,0.017943
7,0.063833,0.032191,0.021812,0.205096,0.016452,0.094633,0.103937,1.000000,0.044922,0.190476,0.058303,0.099845,0.084972,0.071488,0.022123,0.018960,0.026083,0.026552,0.0,0.062610
8,0.079022,0.091088,0.017989,0.159155,0.136409,0.062169,0.078656,0.044922,1.000000,0.106144,0.038153,0.055874,0.119971,0.084185,0.060205,0.031426,0.048263,0.036039,0.0,0.022040
9,0.000000,0.013390,0.027351,0.159606,0.081145,0.055146,0.067860,0.190476,0.106144,1.000000,0.058813,0.121191,0.080279,0.062772,0.063497,0.056480,0.037376,0.010317,0.0,0.033796


In [14]:
# Hàm đề xuất công ty
# # với mỗi sản phẩm, lấy nums sản phẩm tương quan nhất
def get_recommendations(id, cosine_sim=cosine_sim, nums=3):
    idx = df.index[df['id'] == id][0]
    print(idx)
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:nums+1]  # Lấy 3 sản phẩm tương tự nhất
    cp_indices = [i[0] for i in sim_scores]
    return df.iloc[cp_indices]

In [15]:
# Gọi function
recommendations = get_recommendations(68)
recommendations

0


,id,Company Name,Company Type,Company industry,Company size,Country,Working days,Overtime Policy,Company overview,Our key skills,Why you'll love working here,Location,Href,Company_overview,Content
5,45,Blue Belt Technology,IT Product,IT Services and IT Consulting,51-150 employees,Japan,Monday - Friday,Extra salary for OT,Blue Belt Technology is looking for all levels...,Our tech stack\nJava\nNodeJS\nReactJS\nAndroid...,Competitive salary\nPremium health care packag...,"{'Ha Noi': ['Level 3, The Loop Building, 241 X...",https://itviec.com/companies/blue-belt-technol...,blue belt technology looking levels metaverse ...,blue belt technology looking levels metaverse ...
13,460,VNG Corporation,IT Product,Software Products and Web Services,1000+ employees,Vietnam,Monday - Friday,No OT,Build Technologies and Grow People. From Vietn...,Our Key Skills\nJava\nAndroid\niOS\nC++\nQA QC,Attractive salary & benefits you'll love\nBuil...,"{'Ha Noi': ['Phố Nguyễn Chí Thanh, Phường Láng...",https://itviec.com/companies/vng-corporation\n,build technologies grow people vietnam world v...,build technologies grow people vietnam world v...
2,325,OMNIA SYSTEMS COMPANY LIMITED,IT Product,IT Services and IT Consulting,1-50 employees,Sweden,Monday - Friday,No OT,"Omnia Systems Vietnam, the top-ranked digital ...",.NET\nJavaScript\nASP.NET\nReact Native\nC#\nT...,Attractive salary and benefits\nFlexible Worki...,"{'Ho Chi Minh': ['5th floor, 186 Dien Bien Phu...",https://itviec.com/companies/omnia-systems-com...,omnia systems vietnam ranked digital workplace...,omnia systems vietnam ranked digital workplace...


#### Lưu ma trận kết quả consin và đọc lên khi cần đề xuất

In [16]:
# Save cosine_sim to file
import pickle
with open('companies_cosine_sim.pkl', 'wb') as f:
    pickle.dump(cosine_sim, f)

# Open and read file to cosine_sim_new
with open('companies_cosine_sim.pkl', 'rb') as f:
    cosine_sim_new = pickle.load(f)
# cosine_sim_new

In [18]:
def get_recommendations(df, id, cosine_sim=cosine_sim, nums=5):
    # Get the index of the hotel that matches the hotel_id
    matching_indices = df.index[df['id'] == id].tolist()
    if not matching_indices:
        print(f"No company found with ID: {id}")
        return pd.DataFrame()  # Return an empty DataFrame if no match
    idx = matching_indices[0]

    # Get the pairwise similarity scores of all companies with that hotel
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the companies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar companies (Ignoring the company itself)
    sim_scores = sim_scores[1:nums+1]

    # Get the hotel indices
    cp_indices = [i[0] for i in sim_scores]

    # Return the top n most similar hotels as a DataFrame
    return df.iloc[cp_indices]

In [21]:
# Example usage (make sure to use a valid ma_san_pham from your DataFrame)
recommendations = get_recommendations(df, 68, cosine_sim=cosine_sim_new, nums=3)

In [22]:
recommendations[["id", "Company Name", "Content"]].head()

,id,Company Name,Content
5,45,Blue Belt Technology,blue belt technology looking levels metaverse ...
13,460,VNG Corporation,build technologies grow people vietnam world v...
2,325,OMNIA SYSTEMS COMPANY LIMITED,omnia systems vietnam ranked digital workplace...
